import zipfile as zf
ROOT_DIR = "/home/sirui/INNOMAUS/runoff"
files = zf.ZipFile("/home/sirui/INNOMAUS/runoff/b.zip", 'r')
files.extractall(ROOT_DIR)
files.close()

In [4]:
from PIL import Image
import os
import tifffile
import numpy as np
from scipy import interpolate

from skimage.transform import resize
from scipy.ndimage import zoom

def resize_image(input_path, output_path, target_size):
    print(input_path)
    img = tifffile.imread(input_path)
    imag = img
    img = Image.fromarray(img)
    img = img.resize((img.size[0] // 6, img.size[1] // 6), Image.NEAREST)
    img = np.array(img)
    tifffile.imsave(output_path, img)
    print(f"Resized {input_path} to {output_path}")
    # except Exception as e:
    #     print(f"Error processing {input_path}: {str(e)}")
def inter(input_path, output_path, target_size):
    array = tifffile.imread(input_path)
    h, w = array.shape
    new_h, new_w = np.floor_divide((h, w), target_size)
    x = np.linspace(0, w - 1, w)
    y = np.linspace(0, h - 1, h)
    new_x = np.linspace(0, w - 1, new_w)
    new_y = np.linspace(0, h - 1, new_h)
    f = interpolate.interp2d(x, y, array, kind='nearest')
    array_down = f(new_x, new_y)
    tifffile.imsave(output_path, array_down)
    print(f"Resized {input_path} to {output_path}")
    # array_down = resize(array, (new_h, new_w), order=1, anti_aliasing=True)
    return array_down

import numpy as np
import tifffile

def find_non_zero_in_kernel(kernel):
    """
    Find the nearest non-zero element in a given kernel (block of pixels).
    If no non-zero value is found, return 0.
    """
    non_zero_elements = kernel[kernel != 0]
    if len(non_zero_elements) > 0:
        # Return the first non-zero element found (or nearest if required)
        return np.mean(non_zero_elements)
    else:
        return 0  # Return zero if no non-zero elements are found

def resize_with_kernel_nearest_non_zero(array, downsample_factor):
    """
    Resize the array by downsampling by `downsample_factor` (e.g., 6x6 blocks).
    Each downsampled pixel is assigned the nearest non-zero element from its kernel.
    """
    h, w = array.shape
    new_h = h // downsample_factor
    new_w = w // downsample_factor
    
    # Initialize the downsampled array
    result = np.zeros((new_h, new_w), dtype=array.dtype)
    
    for i in range(new_h):
        for j in range(new_w):
            # Extract the 6x6 block (or downsample_factor x downsample_factor block)
            y_start = i * downsample_factor
            y_end = min((i + 1) * downsample_factor, h)
            x_start = j * downsample_factor
            x_end = min((j + 1) * downsample_factor, w)
            
            # Get the block of the original array
            kernel = array[y_start:y_end, x_start:x_end]
            
            # Find the nearest non-zero value in the kernel
            result[i, j] = find_non_zero_in_kernel(kernel)
    
    return result

def inter(input_path, output_path, target_size):
    """
    Read the input TIFF file, downsample the array by a given target_size, and save the result.
    Each downsampled pixel is assigned the nearest non-zero element from its corresponding kernel.
    """
    # Read the input array from a tiff file
    array = tifffile.imread(input_path)
    
    # Ensure the target size is a downsampling factor, e.g., (6, 6)
    if isinstance(target_size, (tuple, list)) and len(target_size) == 2:
        downsample_factor = target_size[0]  # Assuming square downsample blocks
    else:
        downsample_factor = target_size  # Single integer value
    
    # Perform the resizing with nearest non-zero from kernels
    array_down = resize_with_kernel_nearest_non_zero(array, downsample_factor)
    num=len(np.where(array_down!=0)[0])
    # Save the resized array to the output path
    tifffile.imsave(output_path, array_down)
    
    print(f"Resized {input_path} to {output_path} using nearest non-zero kernel interpolation",num)
    return array_down



def process_directory(input_directory, output_directory, target_size):

    for root, _, files in os.walk(input_directory):
        for file in files:
            if file.lower().endswith(".tif"):
                name = os.path.basename(root)
                print(name)
                input_path = os.path.join(root, file)
                output_path1 = os.path.join(output_directory, name)
                if not os.path.exists(output_path1):
                    os.makedirs(output_path1)
                output_path = os.path.join(output_path1, file)
                print(output_path)
                #resize_image(input_path, output_path, target_size)
                inter(input_path, output_path, target_size)

if __name__ == "__main__":
    input_directory_runoff = "/home/sirui/INNOMAUS/runoff_ori"
    output_directory_runoff = "/home/sirui/INNOMAUS/runoff"
    input_directory_gt = "/home/sirui/INNOMAUS/b"
    output_directory_gt = "/home/sirui/INNOMAUS/output"
    target_size = 6

    process_directory(input_directory_runoff, output_directory_runoff, target_size)
    process_directory(input_directory_gt, output_directory_gt, target_size)

b_5y
/home/sirui/INNOMAUS/runoff/b_5y/b_5y_runoff_swmm_2100.tif


/tmp/ipykernel_4135743/2835852815.py:97: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(output_path, array_down)


Resized /home/sirui/INNOMAUS/runoff_ori/b_5y/b_5y_runoff_swmm_2100.tif to /home/sirui/INNOMAUS/runoff/b_5y/b_5y_runoff_swmm_2100.tif using nearest non-zero kernel interpolation 222
b_5y
/home/sirui/INNOMAUS/runoff/b_5y/b_5y_runoff_swmm_3900.tif
Resized /home/sirui/INNOMAUS/runoff_ori/b_5y/b_5y_runoff_swmm_3900.tif to /home/sirui/INNOMAUS/runoff/b_5y/b_5y_runoff_swmm_3900.tif using nearest non-zero kernel interpolation 222
b_5y
/home/sirui/INNOMAUS/runoff/b_5y/b_5y_runoff_swmm_6900.tif
Resized /home/sirui/INNOMAUS/runoff_ori/b_5y/b_5y_runoff_swmm_6900.tif to /home/sirui/INNOMAUS/runoff/b_5y/b_5y_runoff_swmm_6900.tif using nearest non-zero kernel interpolation 220
b_5y
/home/sirui/INNOMAUS/runoff/b_5y/b_5y_runoff_swmm_3600.tif
Resized /home/sirui/INNOMAUS/runoff_ori/b_5y/b_5y_runoff_swmm_3600.tif to /home/sirui/INNOMAUS/runoff/b_5y/b_5y_runoff_swmm_3600.tif using nearest non-zero kernel interpolation 222
b_5y
/home/sirui/INNOMAUS/runoff/b_5y/b_5y_runoff_swmm_1800.tif
Resized /home/sirui/